In [17]:

import numpy as np

import time
import hopsworks
import os
from dotenv import load_dotenv
import joblib
from datetime import datetime, timedelta, date
import pandas as pd

import hopsworks
import matplotlib.pyplot as plt

from utils import *
import utils


In [18]:
load_dotenv()
#os.environ["HOPSWORKS_API_KEY"] = os.getenv("HOPSWORKS_API_KEY")


project = hopsworks.login()

2025-01-02 18:35:57,154 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-02 18:35:57,159 INFO: Initializing external client
2025-01-02 18:35:57,160 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-02 18:35:58,813 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1160346


In [27]:

fs = project.get_feature_store() 
#secrets = secrets_api(project.name)

# This line will fail if you have not registered the CRYPTO_API_KEY as a secret in Hopsworks
#CRYPTO_API_KEY = secrets.get_secret("CRYPTO_API_KEY").value

#CRYPTO_API_KEY = os.getenv("CRYPTO_API_KEY")


today = pd.to_datetime(date.today()).normalize().tz_localize('UTC')


today

Timestamp('2025-01-02 00:00:00+0000', tz='UTC')

In [31]:
#Step 1: get the oldest forcasted date
solana_pred_feature_group = fs.get_feature_group(
    name="solana_predictions",  
    version=1
)

# Step 3: Read the data from the feature group
s_data_df = solana_pred_feature_group.select(["date"]).read()

# Step 4: Find the maximum date
solana_pred_min_date = s_data_df['date'].min()
solana_difference = (today - solana_pred_min_date).days

# Step 5: Print the result
print(f"The maximum date is: {solana_pred_min_date}")
print(f"Difference in days: {solana_difference}")

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.67s) 
The maximum date is: 2025-01-03 00:00:00+00:00
Difference in days: -1


In [45]:
#Step 2: get actual values


url = "https://data-api.cryptocompare.com//spot/v1/historical/days"

# Load environment variables from the .env file
load_dotenv()
crypto_api_key = os.getenv("CRYPTO_API_KEY")


params = {
    "market":"kraken",
    "instrument": "SOL-USD",  
    "limit": solana_difference,  
    "api_key": crypto_api_key,
}
if solana_difference!=0:
    sol_response = trigger_request(url, params)
    sol_response_df = pd.DataFrame(sol_response["Data"])
    sol_response_df = sol_response_df[["TIMESTAMP", 'OPEN']]
    sol_response_df.columns = sol_response_df.columns.str.lower()
    print("Num entries to add: ", len(sol_response_df))
else:
    sol_response_df = pd.DataFrame(columns=["timestamp","open"])


Num entries to add:  2


In [48]:
sol_response_df['date'] = pd.to_datetime(sol_response_df['timestamp'], unit='s')
sol_response_df['date'] = sol_response_df['date'].dt.normalize()
sol_response_df.drop(columns=["timestamp"], inplace=True)
sol_response_df

,open,date
0,188.95,2025-01-01
1,193.82,2025-01-02
